In [ ]:
!wget --no-check-certificate\
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip\
    -O D:/CCNY/Courses/2021-Tensorflow/week1/validation-horse-or-human.zip

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O D:/CCNY/Courses/2021-Tensorflow/week1/horse-or-human.zip

In [ ]:
import os
import zipfile

local_zip = 'D:/CCNY/Courses/2021-Tensorflow/week1/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('D:/CCNY/Courses/2021-Tensorflow/week1/horse-or-human')
zip_ref.close()

In [ ]:
local_zip = 'D:/CCNY/Courses/2021-Tensorflow/week1/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('D:/CCNY/Courses/2021-Tensorflow/week1/validation-horse-or-human')
zip_ref.close()

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 创建两个数据生成器，指定scaling范围0~1
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# 指向训练数据文件夹
train_generator = train_datagen.flow_from_directory(
                'D:/CCNY/Courses/2021-Tensorflow/week1/horse-or-human', # 训练数据所在文件夹
                target_size=(150,150), # 指定输出尺寸
                batch_size=32,
                class_mode='binary') # 指定二分类

# 指向测试数据文件夹
validation_generator = validation_datagen.flow_from_directory(
                'D:/CCNY/Courses/2021-Tensorflow/week1/validation-horse-or-human',
                target_size=(150,150),
                batch_size=32,
                class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
# 构建和训练模型

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
    # first convolution
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # second convolution
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # third convolution
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512,activation='relu'),
    # only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
history = model.fit(
        train_generator,
        epochs=15,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=8)

In [ ]:
# 优化模型的参数

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import tensorflow as tf

hp=HyperParameters()
def build_model(hp):
    model = tf.keras.models.Sequential()
    # first convolution
    model.add(tf.keras.layers.Conv2D(hp.Choice('num_filters_layer0',values=[16,64],default=16),(3,3),activation='relu',input_shape=(150,150,3)))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    # second convolution
    for i in range(hp.Int("num_conv_layers",1,3)): # 数据不能太大，maxpooling之后，输出尺寸会越来越小
        model.add(tf.keras.layers.Conv2D(hp.Choice(f'num_filters_layer{i}',values=[16,64],default=16),(3,3),activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2,2))
    # flatten the results to feed into a DNN
    model.add(tf.keras.layers.Flatten())
    # 512 neuron hidden layer
    model.add(tf.keras.layers.Dense(hp.Int("hidden_units",128,512,step=32),activation='relu'))
    # only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
    return model

tuner=Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=10,
    directory='horse_human_params',
    hyperparameters=hp,
    project_name='my_horse_human_project'
)
tuner.search(train_generator, epochs=10, validation_data=validation_generator)
#model.fit(train_generator, epochs=10, validation_data=validation_generator)

INFO:tensorflow:Reloading Oracle from existing project horse_human_params\my_horse_human_project\oracle.json
INFO:tensorflow:Reloading Tuner from horse_human_params\my_horse_human_project\tuner0.json
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/2
33/33 [==============================] - ETA: 50s - loss: 0.6945 - acc: 0.50 - ETA: 36s - loss: 0.7063 - acc: 0.51 - ETA: 34s - loss: 0.7020 - acc: 0.51 - ETA: 31s - loss: 0.6896 - acc: 0.53 - ETA: 29s - loss: 0.6910 - acc: 0.53 - ETA: 28s - loss: 0.8448 - acc: 0.53 - ETA: 27s - loss: 0.8209 - acc: 0.53 - ETA: 26s - loss: 0.8050 - acc: 0.51 - ETA: 25s - loss: 0.7911 - acc: 0.51 - ETA: 23s - loss: 0.7791 - acc: 0.52 - ETA: 22s - loss: 0.7688 - acc: 0.52 - ETA: 21s - loss: 0.7587 - acc: 0.54 - ETA: 20s - loss: 0.7486 - acc: 0.56 - ETA: 19s - loss: 0.7383 - acc: 0.58 - ETA: 18s - loss: 0.7204 - acc: 0.59 - ETA: 17s - loss: 0.7404 - acc: 0.58 - ETA: 16s - loss: 0.7303 - acc: 0.59 - ETA: 14s - l

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/2
33/33 [==============================] - ETA: 34s - loss: 0.6922 - acc: 0.62 - ETA: 22s - loss: 0.7551 - acc: 0.56 - ETA: 18s - loss: 0.7949 - acc: 0.50 - ETA: 16s - loss: 0.7660 - acc: 0.50 - ETA: 14s - loss: 0.7417 - acc: 0.59 - ETA: 13s - loss: 0.7259 - acc: 0.57 - ETA: 13s - loss: 0.6983 - acc: 0.59 - ETA: 12s - loss: 0.6649 - acc: 0.63 - ETA: 12s - loss: 0.6750 - acc: 0.62 - ETA: 11s - loss: 0.7011 - acc: 0.60 - ETA: 10s - loss: 0.6793 - acc: 0.63 - ETA: 10s - loss: 0.6516 - acc: 0.65 - ETA: 9s - loss: 0.6236 - acc: 0.6779 - ETA: 9s - loss: 0.5899 - acc: 0.700 - ETA: 8s - loss: 0.5639 - acc: 0.716 - ETA: 8s - loss: 0.5438 - acc: 0.728 - ETA: 7s - loss: 0.5458 - acc: 0.724 - ETA: 7s - loss: 0.5249 - acc: 0.739 - ETA: 6s - loss: 0.5043 - acc: 0.751 - ETA: 5s - loss: 0.4796 - acc: 0.752 - ETA: 5s - loss: 0.4653 - acc: 0.762 - ETA: 4s - loss: 0.4477 - acc: 0.773 - ETA: 4s - loss: 0.430

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/2
33/33 [==============================] - ETA: 55s - loss: 0.6960 - acc: 0.50 - ETA: 43s - loss: 13.5721 - acc: 0.421 - ETA: 39s - loss: 10.8644 - acc: 0.447 - ETA: 36s - loss: 8.4063 - acc: 0.429 - ETA: 33s - loss: 6.8839 - acc: 0.41 - ETA: 31s - loss: 5.8494 - acc: 0.43 - ETA: 30s - loss: 5.1067 - acc: 0.43 - ETA: 28s - loss: 4.5602 - acc: 0.42 - ETA: 27s - loss: 4.1464 - acc: 0.42 - ETA: 25s - loss: 3.8186 - acc: 0.44 - ETA: 25s - loss: 3.5284 - acc: 0.46 - ETA: 24s - loss: 3.2849 - acc: 0.48 - ETA: 23s - loss: 3.0703 - acc: 0.52 - ETA: 22s - loss: 2.8802 - acc: 0.54 - ETA: 21s - loss: 2.7501 - acc: 0.54 - ETA: 20s - loss: 2.6081 - acc: 0.54 - ETA: 19s - loss: 2.4802 - acc: 0.55 - ETA: 18s - loss: 2.3597 - acc: 0.57 - ETA: 16s - loss: 2.3430 - acc: 0.56 - ETA: 15s - loss: 2.2449 - acc: 0.58 - ETA: 14s - loss: 2.1534 - acc: 0.60 - ETA: 13s - loss: 2.0675 - acc: 0.61 - ETA: 11s - loss: 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/2
33/33 [==============================] - ETA: 41s - loss: 0.6899 - acc: 0.50 - ETA: 26s - loss: 0.7172 - acc: 0.42 - ETA: 21s - loss: 0.7099 - acc: 0.45 - ETA: 19s - loss: 0.7039 - acc: 0.46 - ETA: 17s - loss: 0.6966 - acc: 0.46 - ETA: 14s - loss: 0.6749 - acc: 0.47 - ETA: 13s - loss: 0.7051 - acc: 0.49 - ETA: 12s - loss: 0.7070 - acc: 0.50 - ETA: 11s - loss: 0.6965 - acc: 0.55 - ETA: 11s - loss: 0.6846 - acc: 0.57 - ETA: 10s - loss: 0.6715 - acc: 0.59 - ETA: 9s - loss: 0.6611 - acc: 0.6197 - ETA: 9s - loss: 0.6517 - acc: 0.620 - ETA: 8s - loss: 0.6445 - acc: 0.627 - ETA: 8s - loss: 0.6331 - acc: 0.643 - ETA: 7s - loss: 0.6214 - acc: 0.652 - ETA: 7s - loss: 0.6171 - acc: 0.652 - ETA: 6s - loss: 0.6053 - acc: 0.667 - ETA: 6s - loss: 0.5901 - acc: 0.683 - ETA: 5s - loss: 0.5743 - acc: 0.697 - ETA: 5s - loss: 0.5579 - acc: 0.710 - ETA: 4s - loss: 0.5524 - acc: 0.714 - ETA: 4s - loss: 0.543

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/2
33/33 [==============================] - ETA: 50s - loss: 0.7005 - acc: 0.50 - ETA: 39s - loss: 17.7816 - acc: 0.468 - ETA: 35s - loss: 14.6496 - acc: 0.489 - ETA: 33s - loss: 11.1590 - acc: 0.492 - ETA: 31s - loss: 9.0628 - acc: 0.493 - ETA: 29s - loss: 7.6966 - acc: 0.49 - ETA: 28s - loss: 6.7533 - acc: 0.47 - ETA: 28s - loss: 5.9903 - acc: 0.50 - ETA: 25s - loss: 5.3762 - acc: 0.50 - ETA: 24s - loss: 5.0097 - acc: 0.50 - ETA: 23s - loss: 4.6087 - acc: 0.53 - ETA: 23s - loss: 4.3297 - acc: 0.54 - ETA: 22s - loss: 4.1561 - acc: 0.52 - ETA: 21s - loss: 3.9008 - acc: 0.56 - ETA: 20s - loss: 3.6780 - acc: 0.58 - ETA: 19s - loss: 3.4746 - acc: 0.60 - ETA: 18s - loss: 3.2906 - acc: 0.63 - ETA: 16s - loss: 3.1900 - acc: 0.62 - ETA: 15s - loss: 3.0935 - acc: 0.61 - ETA: 14s - loss: 2.9763 - acc: 0.61 - ETA: 13s - loss: 2.8665 - acc: 0.60 - ETA: 12s - loss: 2.7619 - acc: 0.60 - ETA: 11s - loss

...

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 6/15
33/33 [==============================] - ETA: 32s - loss: 0.6963 - acc: 0.46 - ETA: 20s - loss: 0.9051 - acc: 0.56 - ETA: 17s - loss: 0.8861 - acc: 0.56 - ETA: 15s - loss: 0.8283 - acc: 0.57 - ETA: 14s - loss: 0.8131 - acc: 0.56 - ETA: 13s - loss: 0.7863 - acc: 0.57 - ETA: 12s - loss: 0.7751 - acc: 0.55 - ETA: 12s - loss: 0.7587 - acc: 0.55 - ETA: 11s - loss: 0.7428 - acc: 0.58 - ETA: 11s - loss: 0.7256 - acc: 0.62 - ETA: 10s - loss: 0.7067 - acc: 0.64 - ETA: 9s - loss: 0.6917 - acc: 0.6380 - ETA: 9s - loss: 0.6766 - acc: 0.639 - ETA: 8s - loss: 0.6577 - acc: 0.660 - ETA: 8s - loss: 0.6385 - acc: 0.670 - ETA: 7s - loss: 0.6265 - acc: 0.671 - ETA: 7s - loss: 0.6268 - acc: 0.663 - ETA: 6s - loss: 0.6110 - acc: 0.675 - ETA: 6s - loss: 0.6045 - acc: 0.677 - ETA: 5s - loss: 0.5925 - acc: 0.685 - ETA: 5s - loss: 0.5779 - acc: 0.697 - ETA: 4s - loss: 0.5589 - acc: 0.711 - ETA: 4s - loss: 0.55

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 6/15
33/33 [==============================] - ETA: 34s - loss: 0.6946 - acc: 0.50 - ETA: 22s - loss: 1.6294 - acc: 0.50 - ETA: 18s - loss: 2.2541 - acc: 0.48 - ETA: 16s - loss: 1.8896 - acc: 0.48 - ETA: 14s - loss: 1.6484 - acc: 0.50 - ETA: 13s - loss: 1.4814 - acc: 0.56 - ETA: 13s - loss: 1.3849 - acc: 0.54 - ETA: 12s - loss: 1.2920 - acc: 0.54 - ETA: 11s - loss: 1.2131 - acc: 0.56 - ETA: 10s - loss: 1.1452 - acc: 0.60 - ETA: 10s - loss: 1.0751 - acc: 0.63 - ETA: 9s - loss: 1.0100 - acc: 0.6536 - ETA: 9s - loss: 1.0029 - acc: 0.634 - ETA: 8s - loss: 1.0320 - acc: 0.620 - ETA: 8s - loss: 0.9968 - acc: 0.631 - ETA: 7s - loss: 0.9591 - acc: 0.650 - ETA: 6s - loss: 0.9110 - acc: 0.652 - ETA: 6s - loss: 0.8793 - acc: 0.667 - ETA: 5s - loss: 0.8542 - acc: 0.678 - ETA: 5s - loss: 0.8248 - acc: 0.685 - ETA: 5s - loss: 0.7916 - acc: 0.699 - ETA: 4s - loss: 0.7625 - acc: 0.711 - ETA: 4s - loss: 0.75

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/5
33/33 [==============================] - ETA: 47s - loss: 0.7020 - acc: 0.43 - ETA: 36s - loss: 1.3548 - acc: 0.45 - ETA: 31s - loss: 1.1853 - acc: 0.45 - ETA: 29s - loss: 1.0634 - acc: 0.46 - ETA: 27s - loss: 0.9866 - acc: 0.49 - ETA: 25s - loss: 0.9338 - acc: 0.51 - ETA: 24s - loss: 0.8973 - acc: 0.54 - ETA: 23s - loss: 0.8608 - acc: 0.56 - ETA: 22s - loss: 0.8402 - acc: 0.56 - ETA: 21s - loss: 0.8403 - acc: 0.55 - ETA: 20s - loss: 0.8259 - acc: 0.57 - ETA: 19s - loss: 0.8116 - acc: 0.57 - ETA: 18s - loss: 0.8187 - acc: 0.55 - ETA: 17s - loss: 0.8069 - acc: 0.55 - ETA: 16s - loss: 0.7936 - acc: 0.58 - ETA: 15s - loss: 0.7745 - acc: 0.59 - ETA: 14s - loss: 0.7578 - acc: 0.61 - ETA: 13s - loss: 0.8552 - acc: 0.60 - ETA: 12s - loss: 0.8468 - acc: 0.59 - ETA: 11s - loss: 0.8366 - acc: 0.61 - ETA: 10s - loss: 0.8233 - acc: 0.61 - ETA: 9s - loss: 0.8105 - acc: 0.6281 - ETA: 8s - loss: 0.793

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/5
33/33 [==============================] - ETA: 58s - loss: 0.6858 - acc: 0.62 - ETA: 44s - loss: 7.4285 - acc: 0.56 - ETA: 38s - loss: 5.1780 - acc: 0.58 - ETA: 34s - loss: 4.3394 - acc: 0.54 - ETA: 32s - loss: 3.6127 - acc: 0.53 - ETA: 30s - loss: 3.1220 - acc: 0.55 - ETA: 28s - loss: 2.7732 - acc: 0.54 - ETA: 26s - loss: 2.5107 - acc: 0.53 - ETA: 25s - loss: 2.3030 - acc: 0.58 - ETA: 23s - loss: 2.1357 - acc: 0.59 - ETA: 22s - loss: 1.9933 - acc: 0.60 - ETA: 21s - loss: 1.8747 - acc: 0.59 - ETA: 20s - loss: 1.7680 - acc: 0.62 - ETA: 18s - loss: 1.6715 - acc: 0.64 - ETA: 17s - loss: 1.6035 - acc: 0.63 - ETA: 16s - loss: 1.5337 - acc: 0.64 - ETA: 15s - loss: 1.4586 - acc: 0.66 - ETA: 14s - loss: 1.3882 - acc: 0.68 - ETA: 13s - loss: 1.3766 - acc: 0.67 - ETA: 12s - loss: 1.3397 - acc: 0.66 - ETA: 11s - loss: 1.3052 - acc: 0.67 - ETA: 10s - loss: 1.2682 - acc: 0.68 - ETA: 9s - loss: 1.2315

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/5
10/33 [========>.....................] - ETA: 55s - loss: 0.7102 - acc: 0.50 - ETA: 46s - loss: 0.7151 - acc: 0.51 - ETA: 41s - loss: 2.8833 - acc: 0.54 - ETA: 39s - loss: 2.3385 - acc: 0.51 - ETA: 35s - loss: 2.0174 - acc: 0.51 - ETA: 34s - loss: 1.7939 - acc: 0.55 - ETA: 32s - loss: 1.6320 - acc: 0.58 - ETA: 31s - loss: 1.5311 - acc: 0.55 - ETA: 29s - loss: 1.4378 - acc: 0.56 - ETA: 28s - loss: 1.3660 - acc: 0.5469

In [ ]:
best_hps=tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)
model=tuner.hypermodel.build(best_hps)